# 3070 running hot

In [2]:
print("Hello world")

Hello world


In [4]:
#Check if cuda is available and running on the right gpu
import torch

torch.cuda.is_available()
torch.cuda.get_device_name()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
!python test_data_balanced.py

In [ ]:
!python train_data.py

Preprocess the images for training, so that the Image Transformation does not have to be done for each training attempt.
The Transformation is pretty CPU and Disk I/O heavy.
Loading the preprocessed tensors eliminates that load. Also the tensors are kept in VRAM for training.

In [ ]:
#%run preprocess_transform.py --data_dir ./inputs/training --output_dir ./inputs/transformed_training
#%run preprocess_transform.py --data_dir ./inputs/test_balanced --output_dir ./inputs/transformed_test_balanced

%run preprocess_transform.py --dataset train
%run preprocess_transform.py --dataset test_balanced

good batch_sizes would be denominators of 4800
100 92% GPU, 2.1G VRAM, 5s
200 90% GPU, 2.4G VRAM, 5s
300 93% GPU, 3.1G VRAM, 4s
400 93% GPU, 3.4G VRAM, 4s
600 93% GPU, 4.7G VRAM, 4s
800 93% GPU, 6.0G VRAM, 4s
1200 93% GPU, 7.7G VRAM, 5s

In [ ]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

main_train(
    model_name='mobilenet_v2',
    data_dir_train='inputs/data_train_local',        # Before was "transformed_training" 
    data_dir_test='inputs/data_test_local',
    output_model_dir='outputs',
    batch_size=200,
    num_epochs=1,
    learning_rate=0.0035148759,
    resume=False,
    test=True
)

# %run train.py \
#     --data_dir_train inputs/transformed_training \
#     --batch_size 200 \
#     --num_epoch 10 \
#     --test True \
#     --data_dir_test ./inputs/transformed_test_balanced \
#     --output_model_dir ./outputs

Run this in Terminal

```sh
uv run train.py
    --data_dir_train inputs/transformed_training
    --batch_size 200
    --num_epoch 3000
    --test True
    --data_dir_test inputs/transformed_test_balanced
    --output_model_dir outputs
```

`uv run tensorboard --logdir=runs`

# Running Vision Transformer

In [ ]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

#learning_rates = [0.01, 0.005, 0.001, 0.0001]
#opt = ['sgd', 'adam']
batch = [1000]
for item in batch:
    main_train(
        model_name='vit_base_patch16_224',
        data_dir_train='inputs/data_train_local',
        data_dir_test='inputs/data_test_local',
        output_model_dir='outputs_vit',
        num_epochs = 1, 
        learning_rate=0.0001,
        batch_size=item, 
        test_every_x_epochs=3,
        resume=False,
        test=True, 
        metric=False,
        optimizer_type= 'adam', 
        momentum=0.0,
 )



Using device: cpu
Model: vit_base_patch16_224
Hyperparameters: {'learning_rate': 0.0001, 'batch_size': 1000, 'num_epochs': 1, 'test_every_x_epochs': 3}
Train Dataset: Found 4800 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']
Test Dataset: Found 1020 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']


c:\Users\Farmer\Desktop\FarmersEye\farmers-eye-pytorch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using optimizer: Adam
Moving model to cpu
Compiling model


c:\Users\Farmer\Desktop\FarmersEye\farmers-eye-pytorch\.venv\Lib\site-packages\torch\amp\grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(



Starting training from epoch 1 to 1
Logging to TensorBoard directory: ./runs\vit_base_patch16_224_20250708_093046
Run 'tensorboard --logdir=runs' to view training progress



Epoch 1/1:   0%|          | 0/4 [00:25<?, ?it/s]


InductorError: RuntimeError: Compiler: cl is not found.

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

main_train(
    model_name='vit_huge_patch14_224',
    data_dir_train='inputs/transformed_training', 
    data_dir_test='inputs/transformed_test_balanced',
    output_model_dir='outputs_vit',
    resume=False,
    test=True
)

```sh

# Foundation Model